In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd gdrive/MyDrive/colab_projects/nlp/imdb/nbs/

/content/gdrive/MyDrive/colab_projects/nlp/imdb/nbs


In [3]:
!pip install transformers

     |████████████████████████████████| 2.0MB 5.7MB/s 
     |████████████████████████████████| 890kB 50.6MB/s 
     |████████████████████████████████| 3.2MB 52.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=ff59b540a32874b4d53455230e59b573acc227ed13f037547643ee6caf056780
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import tensorflow as tf
from transformers import GPT2Tokenizer, GPT2Config, TFGPT2Model

from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical

In [5]:
pd.options.display.max_columns=999
pd.options.display.max_rows=999

In [6]:
df = pd.read_csv("../data/imdb.csv")
df_train = df[:25000]
df_test = df[25000:]

labels_index = {'positive':1, 'negative':0}

In [7]:
# Name of the BERT model to use
model_name = 'gpt2-large'
# Max length of tokens
max_length = 300
# Load transformers config and set output_hidden_states to False
config = GPT2Config.from_pretrained(model_name)
config.output_hidden_states = False
# Load BERT tokenizer
tokenizer = GPT2Tokenizer.from_pretrained(pretrained_model_name_or_path=model_name, config=config)
# Load the Transformers BERT model
transformer_model = TFGPT2Model.from_pretrained(model_name, config=config)

All model checkpoint layers were used when initializing TFGPT2Model.

All the layers of TFGPT2Model were initialized from the model checkpoint at gpt2-large.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2Model for predictions without further training.


In [8]:
transformer_model.summary()

Model: "tfgp_t2model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
transformer (TFGPT2MainLayer multiple                  774030080 
Total params: 774,030,080
Trainable params: 774,030,080
Non-trainable params: 0
_________________________________________________________________


In [9]:
len(transformer_model.layers)

1

In [10]:
# Load the MainLayer
gpt2 = transformer_model.layers[0]

In [11]:
from keras.layers import Dense, Input, GlobalMaxPooling1D, Dropout
from keras.layers import Conv1D, MaxPooling1D, Embedding, LSTM, Flatten
from keras.models import Model

In [12]:
# Build your model input
input_ids = Input(shape=(max_length,), name='input_ids', dtype='int32')
inputs = {'input_ids': input_ids}
gpt2_model = gpt2(inputs)[0]
dense_inter = Flatten()(gpt2_model)
logits = Dense(2)(dense_inter)
model = Model(inputs=inputs, outputs=logits)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ids (InputLayer)       [(None, 300)]             0         
_________________________________________________________________
transformer (TFGPT2MainLayer TFBaseModelOutputWithPast 774030080 
_________________________________________________________________
flatten (Flatten)            (None, 384000)            0         
_________________________________________________________________
dense (Dense)                (None, 2)                 768002    
Total params: 774,798,082
Trainable params: 774,798,082
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Set an optimizer
optimizer = Adam(
    learning_rate=1e-05,
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0)
# Set loss and metrics
loss = CategoricalCrossentropy(from_logits=True)
metric = 'accuracy'
# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)

In [14]:
y_train = to_categorical(df_train['sentiment'].map(labels_index))

In [15]:
y_test = to_categorical(df_test['sentiment'].map(labels_index))

In [16]:
tokenizer.pad_token='[PAD]'

In [17]:
# Tokenize the input (takes some time)
X_train = tokenizer(
    text=df_train['review'].to_list(),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = False,
    verbose = True)

In [18]:
# Tokenize the input (takes some time)
X_test = tokenizer(
    text=df_test['review'].to_list(),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = False,
    verbose = True)

In [19]:
X_train

{'input_ids': <tf.Tensor: shape=(25000, 300), dtype=int32, numpy=
array([[ 3198,   286,   262, ...,  6938,   329, 18024],
       [   32,  7932,  1310, ..., 50256, 50256, 50256],
       [   40,  1807,   428, ..., 50256, 50256, 50256],
       ...,
       [   51,  4741,  2879, ...,   898,  2802,   290],
       [   32,   993,  3763, ..., 50256, 50256, 50256],
       [ 1212,  2646,   815, ..., 50256, 50256, 50256]], dtype=int32)>}

In [20]:
y_train

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [1., 0.]], dtype=float32)

In [21]:
model.fit(X_train["input_ids"], y_train, batch_size=2, epochs=2)

Epoch 1/2
12500/12500 [==============================] - 7667s 611ms/step - loss: 0.3798 - accuracy: 0.8894
Epoch 2/2
12500/12500 [==============================] - 7637s 611ms/step - loss: 0.2418 - accuracy: 0.9406


In [22]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ids (InputLayer)       [(None, 300)]             0         
_________________________________________________________________
transformer (TFGPT2MainLayer TFBaseModelOutputWithPast 774030080 
_________________________________________________________________
flatten (Flatten)            (None, 384000)            0         
_________________________________________________________________
dense (Dense)                (None, 2)                 768002    
Total params: 774,798,082
Trainable params: 774,798,082
Non-trainable params: 0
_________________________________________________________________


In [23]:
score, acc = model.evaluate(X_test["input_ids"], y_test)
print('Test accuracy:', acc)

782/782 [==============================] - 1857s 2s/step - loss: 0.3151 - accuracy: 0.9274
Test accuracy: 0.9273599982261658
